In [1]:
import pyspiel
from open_spiel.python.observation import make_observation

In [2]:
game = pyspiel.load_game("leduc_poker")
state = game.new_initial_state()
print(state.legal_actions())

[0, 1, 2, 3, 4, 5]


In [3]:
state.apply_action(0)
state.apply_action(2)

In [4]:
class player_observation_tree:
    
    def __init__(self, game, player_id):
        self.player = player_id
        
        self.observation = make_observation(
        game,
        pyspiel.IIGObservationType(
                                perfect_recall=False,
                                public_info=False,
                                private_info=pyspiel.PrivateInfoType.SINGLE_PLAYER))
        
        
        
    def get_private_observations(self, state):
        self.observation.set_from(state, player=self.player)
        return self.observation.tensor
    
    
class game_tree:
    
    def __init__(self, game):
        self.observation = make_observation(
        game,
        pyspiel.IIGObservationType(
                                perfect_recall=False,
                                public_info=True,
                                private_info=pyspiel.PrivateInfoType.ALL_PLAYERS))
        
        
    def get_all_childs(self, state):
        
        if (state.current_player() == 0):
            player = 0
        else:
            player = 1
        
        actions_child_states = {}
        for action in state.legal_actions():
            child_state = state.child(action)
            self.observation.set_from(child_state, player=player)
            actions_child_states[action] = self.observation.tensor
            
        return actions_child_states
    
    
class public_observation_tree:
    
    def __init__(self, game):
        
        self.observation = make_observation(
        game,
        pyspiel.IIGObservationType(
                                perfect_recall=False,
                                public_info=True,
                                private_info=pyspiel.PrivateInfoType.NONE))
        
        
        
    def get_public_observations(self,state):
        
        if (state.current_player() == 0):
            self.observation.set_from(state, player=0)
            return self.observation.tensor
        
        self.observation.set_from(state, player=1)
        return self.observation.tensor
    

In [5]:
first_player_tree = player_observation_tree(game, 0)
print(first_player_tree.get_private_observations(state))

[1. 0. 1. 0. 0. 0. 0. 0.]


In [6]:
second_player_tree = player_observation_tree(game, 1)
print(first_player_tree.get_private_observations(state))

[1. 0. 1. 0. 0. 0. 0. 0.]


In [7]:
public_observation_tree = public_observation_tree(game)
print(public_observation_tree.get_public_observations(state))

[1. 0. 0. 0. 0. 0. 0. 0. 1. 1.]


In [8]:
game_tree = game_tree(game)
print(game_tree.get_all_childs(state))


{1: array([1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 3., 1.], dtype=float32), 2: array([1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 3., 1.], dtype=float32)}
